# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [12]:
import sys
sys.path.append('..')
import os
import optuna
from tsai.optuna import *
from tsai.basics import load_object
import papermill as pm
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler
import wandb
from swdf.metrics import ValidationMetricsHandler
from swdf.losses import LossFactory

In [13]:
val_metrics = ValidationMetricsHandler 
val_metrics.list()

Metric Name,Description
Precision,"Calculate the precision metric, which measures the ratio of correctly predicted positive observations to the total predicted positives. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Precision score"
Recall,"Calculate the recall metric, which measures the ratio of correctly predicted positive observations to all observations in the actual class. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Recall score"
F1_Score,"Calculate the F1 score, which is the harmonic mean of precision and recall. It is used as a measure of a model’s accuracy on a dataset. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: F1 score"
Accuracy_Score,"Calculate the accuracy score, which is the ratio of correctly predicted observations to the total observations. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Accuracy score"
Specificity,"Calculate the specificity metric, which measures the proportion of true negatives that are correctly identified. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Specificity score"
Negative_Predictive_Value,"Calculate the Negative Predictive Value (NPV), which measures the proportion of true negatives among all negative predictions. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Negative Predictive Value score"
Detected_Outliers_Difference,"Calculate the change in detected outliers (Δ Detected Outliers), representing the number of true outliers not predicted as outliers. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Count of undetected outliers"
AURPC,"Calculate the Area Under the Precision-Recall Curve (AUPRC), a metric used to evaluate the effectiveness of a model in identifying rare, important events (outliers) Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: AUPRC score"
Skewness_Difference,"Calculate the absolute difference in skewness between the actual and predicted values, which measures the asymmetry of the data distribution. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Absolute difference in skewness between y_true and y_pred"
Kurtosis_Difference,"Calculate the absolute difference in kurtosis between the actual and predicted values, which measures the tailedness of the data distribution. Parameters: y_true (torch.Tensor): Actual values tensor of shape (batch_size, variables, horizon) y_pred (torch.Tensor): Predicted values tensor of the same shape as y_true Returns: torch.Tensor: Absolute difference in kurtosis between y_true and y_pred"


In [14]:
losses = list(LossFactory.losses.keys())
primary_losses = losses[:-2] # Primary losses are the ones that do not need another loss to function.

In [15]:
metrics = [
    'R2_Score',
    'Kurtosis_Difference',
    'Skewness_Difference',
    'MSA',
    'SSPB'
]

val_metrics = val_metrics(metrics)


config = AttrDict(
    study_name = 'general_study_extended', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 100, # number of trials
    train_nb = f'{os.getcwd()}/solfsmy_train.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "arch.decomposition": CategoricalDistribution([True, False]),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1), 
        "arch.individual": CategoricalDistribution([True, False]),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": CategoricalDistribution([2, 4, 8, 16, 32]),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
        "lookback": CategoricalDistribution([18, 24, 36, 128, 192]),
        "loss_func": CategoricalDistribution(losses),
    },
    loss_config = {
        "delta": FloatDistribution(0.2, 10., step=0.2),
        "quantile": FloatDistribution(0.1, 0.9, step=0.05),
        "alpha": FloatDistribution(0.1, 0.9, step=0.05),
        "primary_loss": CategoricalDistribution(primary_losses),
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 100,
        "bs": 64,
        "is_optuna_study": True,
        "metrics": metrics,
        "main_metric": 'R2_Score',
    },
    use_wandb = True,
    wandb_mode = 'offline'
)

%store -d best_valid_metrics                                            

config

```json
{ 'extra_params': { 'bs': 64,
                    'is_optuna_study': True,
                    'main_metric': 'R2_Score',
                    'metrics': [ 'R2_Score',
                                 'Kurtosis_Difference',
                                 'Skewness_Difference',
                                 'MSA',
                                 'SSPB'],
                    'n_epoch': 100},
  'loss_config': { 'alpha': FloatDistribution(high=0.9, log=False, low=0.1, step=0.05),
                   'delta': FloatDistribution(high=10.0, log=False, low=0.2, step=0.2),
                   'primary_loss': CategoricalDistribution(choices=('MSE', 'MAE', 'MSLE', 'RMSLE', 'Hubber', 'Quantile', 'wMSE', 'wMAE', 'wMSLE', 'wRMSLE', 'wHubber', 'wQuantile')),
                   'quantile': FloatDistribution(high=0.9, log=False, low=0.1, step=0.05)},
  'n_trials': 100,
  'search_space': { 'arch.attn_dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.d_ff': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.d_model': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.decomposition': CategoricalDistribution(choices=(True, False)),
                    'arch.dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.individual': CategoricalDistribution(choices=(True, False)),
                    'arch.n_heads': CategoricalDistribution(choices=(2, 4, 8, 16, 32)),
                    'arch.n_layers': IntUniformDistribution(high=6, low=1, step=1),
                    'init_weights': CategoricalDistribution(choices=(True, False)),
                    'lookback': CategoricalDistribution(choices=(18, 24, 36, 128, 192)),
                    'loss_func': CategoricalDistribution(choices=('MSE', 'MAE', 'MSLE', 'RMSLE', 'Hubber', 'Quantile', 'wMSE', 'wMAE', 'wMSLE', 'wRMSLE', 'wHubber', 'wQuantile', 'Classification', 'Trended'))},
  'study_name': 'general_study_extended',
  'study_type': 'bayesian',
  'train_nb': '/workspaces/sw-driver-forecaster/dev_nbs/solfsmy_train.ipynb',
  'use_wandb': True,
  'wandb_mode': 'offline'}
```

In [16]:
def create_objective(train_nb, search_space, extra_params=None, use_wandb=False):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)
            if k == 'loss_func':
                if v in ['Hubber', 'wHubber']:
                    pm_parameters['config.loss_config'] = {
                        'delta': trial._suggest('delta', config.loss_config['delta'])
                    }

                elif v in ['Quantile', 'wQuantile']:
                    pm_parameters['config.loss_config'] = {
                        'quantile': trial._suggest('quantile', config.loss_config['quantile'])
                    }
            
                elif v == 'classification':
                    pm_parameters['config.loss_config'] = {
                        'alpha': trial._suggest('alpha', config.loss_config['alpha']),
                        'primary_loss': trial._suggest('primary_loss', config.loss_config['primary_loss'])
                    }

                elif v == 'Trended':
                    pm_parameters['config.loss_config'] = {
                        'primary_loss': trial._suggest('primary_loss', config.loss_config['primary_loss'])
                    }
                else:
                    pm_parameters['config.loss_config'] = {}

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v
                
        # If using wandb, enable that in the training runs, all of them gathered
        # into a group (NOTE: The train nb must have and use these config arguments)
        if use_wandb:
            pm_parameters['config.use_wandb'] = True
            pm_parameters['config.wandb_group'] = config.study_name + '_runs'

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook

        %store -r valid_metrics
        objective_values = val_metrics.get_objective_values(valid_metrics)
        
        return *objective_values,

    return objective

In [17]:

from pathlib import Path
from fastcore.script import *
import joblib
from importlib import import_module
import warnings
warnings.filterwarnings("ignore")

def run_optuna_study(objective, resume=None, study_type=None, multivariate=True, search_space=None, evaluate=None, seed=None, sampler=None, pruner=None, 
                     study_name=None, direction='maximize', n_trials=None, timeout=None, gc_after_trial=False, show_progress_bar=True, 
                     save_study=True, path='optuna', show_plots=True):
    r"""Creates and runs an optuna study.

    Args: 
        objective:          A callable that implements objective function.
        resume:             Path to a previously saved study.
        study_type:         Type of study selected (bayesian, gridsearch, randomsearch). Based on this a sampler will be build if sampler is None. 
                            If a sampler is passed, this has no effect.
        multivariate:       If this is True, the multivariate TPE is used when suggesting parameters. The multivariate TPE is reported to outperform 
                            the independent TPE.
        search_space:       Search space required when running a gridsearch (if you don't pass a sampler).
        evaluate:           Allows you to pass a specific set of hyperparameters that will be evaluated.
        seed:               Fixed seed used by samplers.
        sampler:            A sampler object that implements background algorithm for value suggestion. If None is specified, TPESampler is used during 
                            single-objective optimization and NSGAIISampler during multi-objective optimization. See also samplers.
        pruner:             A pruner object that decides early stopping of unpromising trials. If None is specified, MedianPruner is used as the default. 
                            See also pruners.
        study_name:         Study’s name. If this argument is set to None, a unique name is generated automatically.
        direction:          A sequence of directions during multi-objective optimization.
        n_trials:           The number of trials. If this argument is set to None, there is no limitation on the number of trials. If timeout is also set to 
                            None, the study continues to create trials until it receives a termination signal such as Ctrl+C or SIGTERM.
        timeout:            Stop study after the given number of second(s). If this argument is set to None, the study is executed without time limitation. 
                            If n_trials is also set to None, the study continues to create trials until it receives a termination signal such as 
                            Ctrl+C or SIGTERM.
        gc_after_trial:     Flag to execute garbage collection at the end of each trial. By default, garbage collection is enabled, just in case. 
                            You can turn it off with this argument if memory is safely managed in your objective function.
        show_progress_bar:  Flag to show progress bars or not. To disable progress bar, set this False.
        save_study:         Save your study when finished/ interrupted.
        path:               Folder where the study will be saved.
        show_plots:         Flag to control whether plots are shown at the end of the study.
    """
    
    try: import optuna
    except ImportError: raise ImportError('You need to install optuna to use run_optuna_study')

    # Sampler
    if sampler is None:
        if study_type is None or "bayes" in study_type.lower(): 
            sampler = optuna.samplers.TPESampler(seed=seed, multivariate=multivariate)
        elif "grid" in study_type.lower():
            assert search_space, f"you need to pass a search_space dict to run a gridsearch"
            sampler = optuna.samplers.GridSampler(search_space)
        elif "random" in study_type.lower(): 
            sampler = optuna.samplers.RandomSampler(seed=seed)
    assert sampler, "you need to either select a study type (bayesian, gridsampler, randomsampler) or pass a sampler"

    # Study
    if resume: 
        try:
            study = joblib.load(resume)
        except: 
            print(f"joblib.load({resume}) couldn't recover any saved study. Check the path.")
            return
        print("Best trial until now:")
        print(" Value: ", study.best_trial.value)
        print(" Params: ")
        for key, value in study.best_trial.params.items():
            print(f"    {key}: {value}")
    else: 
        study = optuna.create_study(sampler=sampler, pruner=pruner, study_name=study_name, directions=direction)
    if evaluate: study.enqueue_trial(evaluate)
    try:
        study.optimize(objective, n_trials=n_trials, timeout=timeout, gc_after_trial=gc_after_trial, show_progress_bar=show_progress_bar)
    except KeyboardInterrupt:
        pass

    # Save
    if save_study:
        full_path = Path(path)/f'{study.study_name}.pkl'
        full_path.parent.mkdir(parents=True, exist_ok=True)
        joblib.dump(study, full_path)
        print(f'\nOptuna study saved to {full_path}')
        print(f"To reload the study run: study = joblib.load('{full_path}')")

    # Plots
    if show_plots and len(study.trials) > 1:
        try: display(optuna.visualization.plot_optimization_history(study))
        except: pass
        try: display(optuna.visualization.plot_param_importances(study))
        except: pass
        try: display(optuna.visualization.plot_slice(study))
        except: pass
        try: display(optuna.visualization.plot_parallel_coordinate(study))
        except: pass

    # Study stats
    try:
        pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
        complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        print(f"\nStudy statistics    : ")
        print(f"  Study name        : {study.study_name}")
        print(f"  # finished trials : {len(study.trials)}")
        print(f"  # pruned trials   : {len(pruned_trials)}")
        print(f"  # complete trials : {len(complete_trials)}")
        
        print(f"\nBest trial          :")
        trial = study.best_trial
        print(f"  value             : {trial.value}")
        print(f"  best_params = {trial.params}\n")
    except:
        print('\nNo finished trials yet.')
    return study


In [18]:
import nbformat

def filter_nb (path:str, skip_tags:list):
    # Load the notebook
    nb = nbformat.read(path, as_version=4)

    # Filter out cells with specific tags
    filtered_cells = [cell for cell in nb.cells if not set(skip_tags) & set(cell.metadata.get('tags', []))]
    nb.cells = filtered_cells
    
    return nb

In [19]:
# Skip the cells that are unnecesary as diagram plots or data update
train_nb = filter_nb(config.train_nb, ['skip'])

obj = create_objective(config.train_nb, config.search_space, 
                       extra_params=config.extra_params, use_wandb=True)
study = run_optuna_study(obj, study_type='bayesian', direction=val_metrics.get_study_directions(), path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2024-08-22 11:31:54,867] A new study created in memory with name: general_study_extended
  0%|          | 0/100 [00:00<?, ?it/s]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.loss_func
Passed unknown parameter: config.loss_config
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.metrics
Passed unknown parameter: config.main_metric
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


  1%|          | 1/100 [01:11<1:58:37, 71.89s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.loss_func
Passed unknown parameter: config.loss_config
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.metrics
Passed unknown parameter: config.main_metric
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-08-22 11:33:06,762] Trial 0 finished with values: [0.45653942227363586, 0.3931412100791931, -10.686614990234375, 3.6878726482391357, -1.1963400840759277] and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 160, 'arch.d_ff': 32, 'arch.decomposition': True, 'arch.dropout': 0.4, 'arch.individual': True, 'arch.n_layers': 5, 'arch.n_heads': 8, 'init_weights': False, 'lookback': 24, 'loss_func': 'wMSE'}. 


  2%|▏         | 2/100 [01:55<1:30:22, 55.33s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.loss_func
Passed unknown parameter: config.loss_config
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.metrics
Passed unknown parameter: config.main_metric
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


[I 2024-08-22 11:33:50,508] Trial 1 finished with values: [0.4522152543067932, 0.3960704803466797, -7.5692362785339355, 4.543078899383545, -1.0914456844329834] and parameters: {'arch.attn_dropout': 0.0, 'arch.d_model': 192, 'arch.d_ff': 256, 'arch.decomposition': True, 'arch.dropout': 0.30000000000000004, 'arch.individual': False, 'arch.n_layers': 1, 'arch.n_heads': 8, 'init_weights': True, 'lookback': 128, 'loss_func': 'Classification'}. 


...

 99%|█████████▉| 99/100 [2:51:34<02:05, 125.17s/it]

[I 2024-08-22 14:23:29,197] Trial 98 finished with values: [0.5159504413604736, 0.4283578097820282, -36.787837982177734, 7.7421770095825195, -3.6638457775115967] and parameters: {'arch.attn_dropout': 0.30000000000000004, 'arch.d_model': 448, 'arch.d_ff': 256, 'arch.decomposition': True, 'arch.dropout': 0.4, 'arch.individual': True, 'arch.n_layers': 5, 'arch.n_heads': 8, 'init_weights': False, 'lookback': 128, 'loss_func': 'wQuantile'}. 


Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.loss_func
Passed unknown parameter: config.loss_config
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.is_optuna_study
Passed unknown parameter: config.metrics
Passed unknown parameter: config.main_metric
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group
100%|██████████| 100/100 [2:53:03<00:00, 103.84s/it]


[I 2024-08-22 14:24:58,519] Trial 99 finished with values: [0.4602906405925751, 0.3709999918937683, -447.4930419921875, 9.788040161132812, -5.007354259490967] and parameters: {'arch.attn_dropout': 0.2, 'arch.d_model': 480, 'arch.d_ff': 64, 'arch.decomposition': True, 'arch.dropout': 0.4, 'arch.individual': True, 'arch.n_layers': 6, 'arch.n_heads': 8, 'init_weights': True, 'lookback': 36, 'loss_func': 'MSLE'}. 

Optuna study saved to tmp/general_study_extended.pkl
To reload the study run: study = joblib.load('tmp/general_study_extended.pkl')



Study statistics    : 
  Study name        : general_study_extended
  # finished trials : 100
  # pruned trials   : 0
  # complete trials : 99

Best trial          :

No finished trials yet.


In [20]:
run = wandb.init(config=config, mode=config.wandb_mode, 
                 job_type='optuna-study') if config.use_wandb else None

In [21]:
if run is not None:
    # Log best trials and their corresponding values
    best_trials_info = {}
    for i, trial in enumerate(study.best_trials):
        best_trials_info[f'best_params_{i}'] = trial.params
        best_trials_info[f'best_value_{i}'] = trial.values
        best_trials_info[f'best_trial_number_{i}'] = trial.number
    
    run.log(best_trials_info)
    run.log_artifact(f'./tmp/{config.study_name}.pkl', type='optuna_study')
    
    # Log visualizations
    for i in range(len(study.directions)):  # Assuming each objective has a direction
            run.log({
                f'contour_obj_{i}': optuna.visualization.plot_contour(study, target=lambda t: t.values[i], target_name=f'Objective {i}'),
                f'edf_obj_{i}': optuna.visualization.plot_edf(study, target=lambda t: t.values[i], target_name=f'Objective {i}'),
                f'optimization_history_obj_{i}': optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[i], target_name=f'Objective {i}'),
                f'parallel_coordinate_obj_{i}': optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[i], target_name=f'Objective {i}'),
                f'param_importances_obj_{i}': optuna.visualization.plot_param_importances(study, target=lambda t: t.values[i], target_name=f'Objective {i}'),
                f'slice_obj_{i}': optuna.visualization.plot_slice(study, target=lambda t: t.values[i], target_name=f'Objective {i}')
            })


In [22]:
if run is not None:
    run.finish()

best_trial_number_0,▁
best_trial_number_1,▁
best_trial_number_10,▁
best_trial_number_11,▁
best_trial_number_12,▁
best_trial_number_13,▁
best_trial_number_14,▁
best_trial_number_15,▁
best_trial_number_16,▁
best_trial_number_17,▁
best_trial_number_18,▁
